## Reto: Movilidad Urbana
- Alejandro Enríquez Coronado A01378141
- Isaac Cortés Martínez	A01378642
- Claudia Sarahí Armenta Maya  A01378067
- Ariel Álvarez Cortés  A01378734


In [11]:
# Mesa
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

# Matplotlib
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Libraries
import numpy as np
import pandas as pd
import random
import os

# Functions

In [12]:
def load_city(path):
    """
    Function that loads the city as a text file into the model
    and returns the height, width and a list with every parsed
    line in the file.
    """
    with open(path, encoding='utf8') as f:
        city = str()
        for line in f:
            city += line
    width = len(city.partition('\n')[0])
    city = [x for x in city.split('\n')]
    height = len(city)
    
    return height, width, city

def get_grid(model):
    """
    Stores the grid content in an array.
    """
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        grid[x][y] = cell_content[0].color
        if len(cell_content)>1:
            for agent in cell_content:
                if isinstance(agent, Car):
                    grid[x][y] = agent.color
    return grid

# Car
Agent that moves to a destiny. Stops in a stoplight and avoids crashing with other cars.

In [13]:
class Car(Agent):
    def __init__(self, unique_id, model, destinyCoords):
        super().__init__(unique_id, model)
        self.color = 1
        self.moves = 0
        self.direction = None
        self.semaphore =  False
        self.destiny = destinyCoords

    def move(self):
        self.update_direction()
        nc = self.get_neighborhood_content()
        cars = self.get_neighborhood_cars()
        center = self.model.grid.get_cell_list_contents(self.pos)
        y, x  = self.pos
        yd, xd = self.destiny
        
        if(type(self.get_far_neighborhood_content()) == list):
            nc = nc + self.get_far_neighborhood_content()
        
        if(self.semaphore == True):
            pass
        elif((y,x+1) == (yd, xd) or (y,x-1) == (yd, xd) or (y+1,x) == (yd, xd) or (y-1,x) == (yd, xd)):
            self.model.grid.move_agent(self, (yd,xd))
        elif(self.direction == "South" and (y,x+2) == (yd,xd)):
            self.model.grid.move_agent(self, (y,x+1))
    
        elif(self.direction == "East" and  len(nc) > 6 and nc[6] == "v"  and self.check_proximity(4)):
            self.check_turn(1)
        elif(self.direction == "East" and  len(nc) > 7 and ( nc[-2] == "v")  and self.check_proximity(4)):
            self.check_turn(1)
        elif(self.direction == "East" and  len(nc) > 7 and ( nc[-1] == "^")  and self.check_proximity(1)):
            self.check_turn(4)
        elif(self.direction == "East"):
            
            if (len(cars) > 0 and cars[0].pos[1] == x+1):
                pass
            else:
                self.model.grid.move_agent(self, (y,x+1))
                
        elif(self.direction == "North" and  len(nc) > 6 and nc[4] == ">"  and (13,15) == (yd,xd)):
            self.check_turn(2)
        elif(self.direction == "North" and  len(nc) > 6 and nc[4] == ">"  and self.check_proximity(3)):
            self.check_turn(2)
        elif(self.direction == "North" and  len(nc) > 7 and (nc[-2] == ">" )  and self.check_proximity(3)):
            self.check_turn(2)
        elif(self.direction == "North" and  len(nc) > 7 and (nc[-1] == "<" )  and self.check_proximity(2)):
            self.check_turn(3)
        elif(self.direction == "North"):
            
            if (len(cars) > 0 and cars[0].pos[0] == y-1):
                pass
            else:
                self.model.grid.move_agent(self, (y-1,x))
                
        elif(self.direction == "South" and  len(nc) > 6 and nc[3] == "<"  and self.check_proximity(2)):
            self.check_turn(3)
        elif(self.direction == "South" and  len(nc) > 7 and (nc[-1] == "<" )  and self.check_proximity(2)):
            self.check_turn(3)
        elif(self.direction == "South" and  len(nc) > 7 and (nc[-2] == ">")  and self.check_proximity(3)):
            self.check_turn(2)
        elif(self.direction == "South"):
            
            if (len(cars) > 0 and cars[0].pos[0] == y+1):
                pass
            else:
                self.model.grid.move_agent(self, (y+1,x))
                
        elif(self.direction == "West" and  len(nc) > 6 and nc[1] == "^"  and self.check_proximity(1)):
            self.check_turn(4)
        elif(self.direction == "West" and  len(nc) > 7 and (nc[-1] == "^" )  and self.check_proximity(1)):
            self.check_turn(4)
        elif(self.direction == "West" and  len(nc) > 7 and (nc[-2] == "v" )  and self.check_proximity(3)):
            self.check_turn(1)
        elif(self.direction == "West"):
            
            if (len(cars) > 0 and cars[0].pos[1] == x-1):
                pass
            else:  
                self.model.grid.move_agent(self, (y,x-1))
                
        elif(self.direction == "Destiny"):
            pass
        elif(self.direction == "Stop"):
            pass
        else:
            pass
        self.moves += 1
        
    def check_turn(self, case):
        nc = self.get_neighborhood_content()
        y, x  = self.pos
        if(case == 1):
            self.model.grid.move_agent(self, (y+1,x))
        elif(case == 2):
            self.model.grid.move_agent(self, (y,x+1))
        elif(case == 3):
            self.model.grid.move_agent(self, (y,x-1))
        elif(case == 4):
            self.model.grid.move_agent(self, (y-1,x))
      
    def check_proximity(self, case):
        y, x  = self.pos
        yd, xd = self.destiny
        # North
        if(case == 1):
            if(xd >= x-1):
                return True
        # East
        elif(case == 2):
            if(yd <= y-1):
                return True
        # West
        elif(case == 3):
            if(yd >= y+1):
                return True
        # South
        elif(case == 4):
            if(xd <= x+1):
                return True
        else:
            return False

    def update_direction(self):
        nc = self.get_neighborhood_content()
        center = self.model.grid.get_cell_list_contents(self.pos)
        center = [x for x in self.model.grid.get_cell_list_contents(self.pos) if not isinstance(x,Car)][0].symbol
        if (center == "^"):
            self.direction = "North"
        elif (center == "v"):
            self.direction = "South"
        elif (center == '<'):
            self.direction = "West"
        elif (center == '>'):
            self.direction = "East"            
        elif(center == 'D'):
            self.direction = "Destiny"
        elif (center == 's' or center == 'S'):
            sema = self.model.grid.get_cell_list_contents(self.pos)
            if(sema[0].stoplight == False):
                self.semaphore = False
            else:
                self.semaphore = True
                
    def get_neighborhood_content(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos,
        moore=True,
        include_center=False)
        # Check content of possible steps
        possible_steps_content = list()
        for i in range (len(possible_steps)):
            a = self.model.grid.get_cell_list_contents([possible_steps[i]])[0]
            possible_steps_content.append(a.symbol)
            
        return possible_steps_content
    
    def get_neighborhood_cars(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos,
        moore=False,
        include_center=False)
        # Check content of possible steps
        possible_steps_content = list()
        for i in range (len(possible_steps)):
            a = self.model.grid.get_cell_list_contents([possible_steps[i]])
            for agent in a:
                if (isinstance(agent,Car)):
                    possible_steps_content.append(agent)
        return possible_steps_content
    
    def get_far_neighborhood_content(self):
        # Para cada caso
        # North: x+2, x-2 a menos que esté en borde
        # South  x+2, x-2 a menos que esté en borde
        y,x = self.pos
        a = []
        if(self.direction == "North" or self.direction == "South"):
            if(x < 2):
                coordenada = (y, x+2)
                possible_step = self.model.grid.get_neighborhood(coordenada,moore=True,include_center=True)
                a.append(self.model.grid.get_cell_list_contents([possible_step[4]])[0].symbol)
                a.append(0)
                return(a)
            elif (x > self.model.width - 3):
                coordenada = (y, x-2)
                possible_step = self.model.grid.get_neighborhood(coordenada,moore=True,include_center=True)
                a.append(0)
                a.append(self.model.grid.get_cell_list_contents([possible_step[4]])[0].symbol)
                return(a)
            
            else:
                coordenada1 = (y, x-2)
                coordenada2 = (y, x+2)
                possible_step2 = self.model.grid.get_neighborhood(coordenada2,moore=True,include_center=True) 
                possible_step1 = self.model.grid.get_neighborhood(coordenada1,moore=True,include_center=True)
                a.append(self.model.grid.get_cell_list_contents([possible_step1[4]])[0].symbol)
                a.append(self.model.grid.get_cell_list_contents([possible_step2[4]])[0].symbol)
                return(a)
        # East y+2, y-2 a menos que esté en borde
        # West y+2, y-2 a menos que esté en borde
        elif (self.direction == "East" or self.direction == "West"):
            if(y < 2):
                coordenada = (y+2,x)
                possible_step = self.model.grid.get_neighborhood(coordenada,moore=True,include_center=True)
                a.append(self.model.grid.get_cell_list_contents([possible_step[4]])[0].symbol)
                a.append(0)
                return(a)
            elif (y > self.model.height - 3):
                coordenada = (y-2, x)
                possible_step = self.model.grid.get_neighborhood(coordenada,moore=True,include_center=True)
                a.append(0)
                a.append(self.model.grid.get_cell_list_contents([possible_step[4]])[0].symbol)
                return(a)
            else:
                coordenada1 = (y-2, x)
                coordenada2 = (y+2, x)
                possible_step1 = self.model.grid.get_neighborhood(coordenada1,moore=True,include_center=True)
                possible_step2 = self.model.grid.get_neighborhood(coordenada2,moore=True,include_center=True)
                a.append(self.model.grid.get_cell_list_contents([possible_step2[4]])[0].symbol)
                a.append(self.model.grid.get_cell_list_contents([possible_step1[4]])[0].symbol)
                return(a)
        
    def step(self):
        self.move()

# Tile
Objects that represent a piece of road, building, destiny, etc.

In [14]:
class Tile(Agent):
    def __init__(self, unique_id, symbol, model):
        super().__init__(unique_id, model)
        self.symbol = symbol
        self.color = model.color_dic[self.symbol]       
        
    def step(self):
        pass

# Stoplight
Object that represents a stoplight that changes from "Red" to "Green" every determined number of steps.

In [15]:
class Stoplight(Agent):
    def __init__(self, unique_id, symbol, stoplight, model):
        super().__init__(unique_id, model)
        self.symbol = symbol
        self.color = model.color_dic[self.symbol]
        self.stoplight = stoplight
        self.count = 0
        
    def step(self):
        self.count += 1
        
        if self.count == 10:
            self.count = 0
            self.stoplight = not self.stoplight
        
        if self.stoplight == True:
            self.color = 3
        elif self.stoplight == False:
            self.color = 9

# Model

In [16]:
class ArrangementModel(Model):
    def __init__(self, city, N, width, height):
        # Initialize model parameters
        self.city = city
        self.num_agents = N
        self.height = height
        self.width = width
        self.color_dic = {
            "D": 2,
            "S": 3,
            "s": 3,
            "^": 4,
            ">": 5,
            "v": 6,
            "<": 7,
            "#": 8
        }
        self.destinies = []
        self.street_tiles = []
        # Set up model objects
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)
        
        # Create the city
        for (content, x, y) in self.grid.coord_iter():
            if (self.city[x][y] == 'D'):
                self.destinies.append((x,y))
            elif (self.city[x][y] in ('^><v')):
                self.street_tiles.append((x,y))
            tmp = self.city[x][y]
        
            if (tmp in 'Ss'):
                if (tmp == 'S'):
                    a = Stoplight(str((x,y)), tmp, True, self)
                else:
                    a = Stoplight(str((x,y)), tmp, False, self)
            else:
                a = Tile(str((x,y)), tmp, self)
                
            self.schedule.add(a)
            self.grid.place_agent(a, (x, y))
        
        # Create cars
        for j in range(self.num_agents):
            a = Car('Car-'+str(j), self, random.choice(self.destinies))
            self.schedule.add(a)
            spawn = random.choice(self.street_tiles)
            self.grid.place_agent(a, spawn)
            self.street_tiles.pop(self.street_tiles.index(spawn))
              
        # Define collector for the entire grid 
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
        # Run the model until it is halted
        self.running = True
        
    def step(self):
        self.datacollector.collect(self)
        # Halt if no cars in the map
        if self.count_in_destiny() == self.num_agents:
            self.running = False
        self.schedule.step()
    
    def count_in_destiny(self):
        count = 0
        for agent in self.schedule.agents:
            if (isinstance(agent,Car)):
                if (agent.direction == "Destiny"):
                    count += 1
        return count

# Correr

In [17]:
path = os.path.abspath(os.path.join(os.getcwd(),'city.txt'))

# Define params 
agents = 30
tmax = 10
height, width, city = load_city(path)
model = ArrangementModel(city, agents, height, width)

model.run_model()

In [18]:
all_grid = model.datacollector.get_model_vars_dataframe()
len(all_grid)

131

In [19]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.get_cmap("Accent"))

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [20]:
anim